In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import subprocess
import requests
from threading import Thread
import socks
import socket
from cryptography.fernet import Fernet

# Generate a key for encryption
key = Fernet.generate_key()
cipher_suite = Fernet(key)

class TorToolsApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Tor Tools")
        self.root.geometry('400x400')

        self.create_widgets()

    def create_widgets(self):
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Button to refresh Tor connection
        self.refresh_button = ttk.Button(self.main_frame, text="Refresh Tor Connection", command=self.refresh_tor_connection)
        self.refresh_button.pack(pady=20)

        # Button to check Tor connection status
        self.check_status_button = ttk.Button(self.main_frame, text="Check Tor Connection Status", command=self.check_tor_status)
        self.check_status_button.pack(pady=10)

        # Search bar for performing searches
        self.search_var = tk.StringVar()
        self.search_bar = ttk.Entry(self.main_frame, textvariable=self.search_var)
        self.search_bar.pack(pady=10)
        self.search_bar.bind('<Return>', self.perform_search)

        # Text widget to display current IP address and other information
        self.onion_text = tk.Text(self.main_frame, height=5, width=30)
        self.onion_text.pack(pady=10)

        # Button to enable Tor proxy
        self.proxy_button = ttk.Button(self.main_frame, text="Enable Tor Proxy", command=self.enable_tor_proxy)
        self.proxy_button.pack(pady=10)

    def refresh_tor_connection(self):
        try:
            subprocess.Popen(["open", "-a", "Tor Browser"])  # For macOS
            messagebox.showinfo("Success", "Tor browser launched.")
        except FileNotFoundError:
            messagebox.showerror("Error", "Tor Browser not found. Please install it first.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def check_tor_status(self):
        self.onion_text.delete('1.0', tk.END)
        loading_text = "Checking Tor Connection Status..."
        self.onion_text.insert(tk.END, loading_text)

        # Use threading to prevent GUI freezing
        t = Thread(target=self.get_tor_status)
        t.start()

    def get_tor_status(self):
        try:
            self.setup_tor_proxy()
            current_ip = requests.get('https://api.ipify.org').text
            encrypted_ip = cipher_suite.encrypt(current_ip.encode()).decode()
            result_text = f"Current IP Address: {encrypted_ip}\nAnonymity Level: High"  # Example
            self.onion_text.delete('1.0', tk.END)
            self.onion_text.insert(tk.END, result_text)
        except requests.RequestException as e:
            messagebox.showerror("Error", f"Request Exception: {e}")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def perform_search(self, event):
        term = self.search_var.get().strip()
        if term:
            encoded_term = cipher_suite.encrypt(term.encode()).decode()
            decrypted_term = cipher_suite.decrypt(encoded_term.encode()).decode()
            self.open_website(f"https://search.brave.com/search?q={decrypted_term}")
        # Clear the search bar
        self.search_var.set("")

    def open_website(self, url):
        try:
            self.setup_tor_proxy()
            subprocess.Popen(["open", "-a", "Tor Browser", url])
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def enable_tor_proxy(self):
        try:
            self.setup_tor_proxy()
            messagebox.showinfo("Success", "Tor proxy enabled.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def setup_tor_proxy(self):
        socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)  # Update the port number to the appropriate Tor port
        socket.socket = socks.socksocket

def main():
    root = tk.Tk()
    app = TorToolsApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()